In [1]:
import pandas as pd

In [2]:
latest_data = pd.read_csv('latest_data.csv')
latest_data

,Unnamed: 0,date,Open,Close,Adj Close,Company,Industry,Stock ticker,daily return,nyt_pol,nyt_sub
0,0,2015-01-02,129.639999,129.949997,113.439125,Linde plc,Basic Materials,LIN,0.002391,0.072037,0.321540
1,1,2015-01-05,128.820007,126.519997,110.444939,Linde plc,Basic Materials,LIN,-0.017854,0.181212,0.396078
2,2,2015-01-06,126.199997,124.900002,109.030746,Linde plc,Basic Materials,LIN,-0.010301,0.066854,0.361403
3,3,2015-01-07,125.870003,126.300003,110.252884,Linde plc,Basic Materials,LIN,0.003416,0.105118,0.358165
4,4,2015-01-08,127.889999,128.380005,112.068611,Linde plc,Basic Materials,LIN,0.003831,0.083550,0.307433
...,...,...,...,...,...,...,...,...,...,...,...
49018,49018,2019-12-23,20.950001,21.200001,19.417809,Kinder Morgan,Energy,KMI,0.011933,0.144016,0.354293
49019,49019,2019-12-24,21.180000,21.129999,19.353691,Kinder Morgan,Energy,KMI,-0.002361,0.141380,0.359756
49020,49020,2019-12-26,21.190001,21.200001,19.417809,Kinder Morgan,Energy,KMI,0.000472,0.125000,0.329167
49021,49021,2019-12-27,21.260000,21.080000,19.307896,Kinder Morgan,Energy,KMI,-0.008467,0.075000,0.410417


In [3]:
newdata = latest_data[latest_data['date'].str.slice(0,4)=='2019']
newdata

,Unnamed: 0,date,Open,Close,Adj Close,Company,Industry,Stock ticker,daily return,nyt_pol,nyt_sub
1006,1006,2019-01-02,153.529999,157.199997,151.025879,Linde plc,Basic Materials,LIN,0.023904,0.065703,0.304392
1007,1007,2019-01-03,154.059998,152.289993,146.308685,Linde plc,Basic Materials,LIN,-0.011489,0.079167,0.402891
1008,1008,2019-01-04,153.830002,156.630005,150.478256,Linde plc,Basic Materials,LIN,0.018202,0.149583,0.238437
1009,1009,2019-01-07,155.779999,156.490005,150.343765,Linde plc,Basic Materials,LIN,0.004558,0.212921,0.398316
1010,1010,2019-01-08,156.089996,158.149994,151.938553,Linde plc,Basic Materials,LIN,0.013198,0.022998,0.143681
...,...,...,...,...,...,...,...,...,...,...,...
49018,49018,2019-12-23,20.950001,21.200001,19.417809,Kinder Morgan,Energy,KMI,0.011933,0.144016,0.354293
49019,49019,2019-12-24,21.180000,21.129999,19.353691,Kinder Morgan,Energy,KMI,-0.002361,0.141380,0.359756
49020,49020,2019-12-26,21.190001,21.200001,19.417809,Kinder Morgan,Energy,KMI,0.000472,0.125000,0.329167
49021,49021,2019-12-27,21.260000,21.080000,19.307896,Kinder Morgan,Energy,KMI,-0.008467,0.075000,0.410417


In [4]:
import twint
import nest_asyncio
import pprint
from textblob import TextBlob
import random
import time
nest_asyncio.apply()

In [5]:
def twitter_polsub(row):
    ticker = row['Stock ticker']
    date = row['date']

    # Configure
    c = twint.Config()
    query = '$' + ticker + ' OR ' + '$' + ticker.lower()
    c.Search = query
    c.Limit = 20
    c.Popular_tweets = True
    c.Store_object = True
    c.Hide_output = True
    c.Until = date

    # Run
    twint.run.Search(c)
    tweets = twint.output.tweets_list
    
    pol = 0
    sub = 0
    # extracting and storing top 20 tweets as a list
    for t in tweets[-20:]:
        p, s = TextBlob(t.tweet).sentiment
        pol += p
        sub += s
        
    time.sleep(5)
    return pol/20, sub/20

In [6]:
def scrape(df):
    pol = []
    sub = []
    for i in range(0, len(df), 499):
        df1 = df[i:i+117]
        p, s = zip(*df1.apply(lambda row: twitter_polsub(row), axis=1))
        pol.append(list(p))
        sub.append(list(s))
        time.sleep(10)
        print(i)
        
    pol = [x for i in pol for x in i]
    sub = [x for i in sub for x in i]
    lst = {'twt_pol':pol, 'twt_sub':sub}
    return pd.DataFrame(lst)

In [ ]:
df = scrape(newdata) #newdata
df

0
499
998


In [ ]:
def twintfunc(ticker, dates=[]):
    tweet_dict = {}

    for i in range(len(dates) - 2):
        # Configure
        c = twint.Config()
        c.Limit = 20
        c.Popular_tweets = True
        c.Store_object = True
        c.Hide_output = True
        c.Search = "$" + ticker
        c.Since = dates[i]
        c.Until = dates[i + 2]

        # Run
        twint.run.Search(c)
        tweets = twint.output.tweets_list

        tweet_dict[dates[i]] = [t.tweet for t in tweets[:20]]

    return tweet_dict

In [ ]:
ticker = 'aapl'
dates = ['2020-3-16', '2020-3-17', '2020-3-18', '2020-3-19', '2020-3-20']
aapl = twintfunc(ticker, dates)

In [ ]:
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(aapl)